# As ações mais baratas da B3 pela Fórmula de Benjamin Graham

## 1. Objetivo

- Desenvolver um script em Python utilizando a biblioteca Selenium para realizar o web scraping do site [Fundamentus](https://www.fundamentus.com.br/) com o propósito de coletar dados dos ativos listados na B3. Após a coleta, os dados serão tratados e utilizados para identificar os ativos mais baratos da bolsa com base na Fórmula de Benjamin Graham, que leva em consideração métricas como Lucro por Ação (LPA) e Valor Patrimonial por Ação (VPA).


- Vamos combinar o Indicador EV/EBIT e o Valor Intrínseco pela Fórmula de Graham para avaliar a atratividade de um ativo na Bolsa de Valores.


- Um EV/EBIT baixo geralmente indica que a empresa está sendo negociada a um preço atrativo em relação ao seu lucro operacional, sugerindo potencial de valorização.


- Para garantir uma análise conservadora e proteger contra incertezas, aplicaremos margens de segurança de 60%, 50% e 40% ao valor intrínseco calculado.


- A decisão de compra será tomada apenas se o preço de mercado da ação estiver abaixo do valor ajustado pela margem de segurança escolhida. Entretanto, esta não deve ser considerada uma recomendação de compra ou venda de qualquer ativo.

## 2. Coleta de Dados

- Utilizaremos o Selenium em conjunto com o navegador Google Chrome para realizar o web scraping.


- Para a instalar a biblioteca do Selenium abra o prompt do anaconda e execute o comando:
```
pip install --upgrade selenium
```

- O Selenium para funcionar, precisa também do webdriver que no Chrome se chama chromedriver. O webdriver é o que vai permitir o Selenium comandar o navegador.


- O webdriver-manager automatiza a atualização do ChromeDriver, eliminando a necessidade de baixá-lo toda vez que o Google Chrome for atualizado. 
```
pip install webdriver-manager
``` 

### 2.1 Instalação e Configurações Importantes do Selenium

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [29]:
# Para acessar ao site  Fundamentus com o Selenium, usaremos a variável navegador que foi configurada
navegador.get("https://www.fundamentus.com.br/")

### 2.2 Filtrando Ações por P/L e Liquidez com Métodos de Seleção do Selenium

- Como não queremos empresas com lucro negativo, definiremos o P/L (Preço/Lucro) mínimo como 0 e a liquidez diária mínima como R$ 1.000.000, sendo que este último critério pode ser ajustado conforme necessário.

In [30]:
from selenium.webdriver.common.by import By

navegador.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/span/a[1]').click()

navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/table/tbody/tr[1]/td[2]/span/input[1]').click()
navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/table/tbody/tr[1]/td[2]/span/input[1]').send_keys("0")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/table/tbody/tr[16]/td[2]/span/input[1]').send_keys("1000000")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/input').click()

### 2.3 Extraindo as informações  da tabela

In [31]:
# Localizando o elemento tabela pelo ID
table = navegador.find_element(By.ID, 'resultado')

# Identificando todas as linhas da tabela
rows = table.find_elements(By.TAG_NAME, 'tr')

In [32]:
rows[0].text

'Papel Cotação P/L P/VP PSR Div.Yield P/Ativo P/Cap.Giro P/EBIT P/Ativ Circ.Liq EV/EBIT EV/EBITDA Mrg Ebit Mrg. Líq. Liq. Corr. ROIC ROE Liq.2meses Patrim. Líq Dív.Brut/ Patrim. Cresc. Rec.5a'

## 3. Tratamento dos Dados

### 3.1 Editando o cabeçalho da tabela

In [33]:
chaves =rows[0].text

chaves=chaves.split(" ")

chaves


['Papel',
 'Cotação',
 'P/L',
 'P/VP',
 'PSR',
 'Div.Yield',
 'P/Ativo',
 'P/Cap.Giro',
 'P/EBIT',
 'P/Ativ',
 'Circ.Liq',
 'EV/EBIT',
 'EV/EBITDA',
 'Mrg',
 'Ebit',
 'Mrg.',
 'Líq.',
 'Liq.',
 'Corr.',
 'ROIC',
 'ROE',
 'Liq.2meses',
 'Patrim.',
 'Líq',
 'Dív.Brut/',
 'Patrim.',
 'Cresc.',
 'Rec.5a']

In [34]:
chaves[9]= "P/Ativ Circ.Liq"
chaves[13]= "Mrg Ebit"
chaves[15]= "Mrg Líq."
chaves[17]= "Líq. Corr."
chaves[22]= "Patrim. Líq"
chaves[24]= "Dív.Brut/Patrim."
chaves[26]= "Cresc. Rec.5a"

chaves.remove("Circ.Liq")
chaves.remove("Ebit")
chaves.remove("Líq.")
chaves.remove("Corr.")
chaves.remove("Líq")
chaves.remove("Patrim.")
chaves.remove("Rec.5a")

chaves

['Papel',
 'Cotação',
 'P/L',
 'P/VP',
 'PSR',
 'Div.Yield',
 'P/Ativo',
 'P/Cap.Giro',
 'P/EBIT',
 'P/Ativ Circ.Liq',
 'EV/EBIT',
 'EV/EBITDA',
 'Mrg Ebit',
 'Mrg Líq.',
 'Líq. Corr.',
 'ROIC',
 'ROE',
 'Liq.2meses',
 'Patrim. Líq',
 'Dív.Brut/Patrim.',
 'Cresc. Rec.5a']

### 3.2 Criando um loop para iterar sobre cada linha da tabela e armazenar os dados em uma lista

In [35]:
fila=[]

for i in range(1, len(rows)):
    papel=rows[i].text
    papel= papel.split(" ")
    fila.append(papel)

### 3.3 Criando um dicionário com as listas 'chaves' e 'fila' e armazenando-o à lista 'acoes_list'

In [36]:
acoes_list = []

for i in range(0, len(fila)):
    papel_tuplas = zip(chaves,fila[i])
    papel_dict =dict(papel_tuplas)
    acoes_list.append(papel_dict)

### 3.4 Convertendo a Lista "acoes_list" em um DataFrame

In [37]:
import pandas as pd

papel_df = pd.DataFrame(acoes_list)
papel_df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg Líq.,Líq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/Patrim.,Cresc. Rec.5a
0,OIBR3,"1,45","0,04","-0,03","0,055","0,00%","0,023","-0,23","-0,13","-0,02",...,"-4,23","-41,26%","137,65%","0,76","-22,29%","-81,27%","3.136.460,00","-14.802.100.000,00","-0,69","0,71%"
1,AMER3,"5,97","0,11","0,21","0,081","0,00%","0,065","0,39","-0,81","-0,22",...,"-8,80","-10,09%","76,26%","1,70","-9,68%","196,01%","17.350.700,00","5.717.000.000,00","0,30","10,19%"
2,SYNE3,"5,07","1,57","0,49","0,555","129,21%","0,204","0,72","1,49","-5,61",...,"1,56","37,24%","44,92%","11,34","16,74%","31,25%","5.709.650,00","1.573.570.000,00","0,54","21,71%"
3,VBBR3,"16,48","2,01","0,90","0,107","9,53%","0,388","1,26","3,63","-6,98",...,"4,96","2,96%","5,33%","2,50","13,58%","44,82%","202.622.000,00","20.424.000.000,00","0,84","20,31%"
4,POSI3,"5,43","2,96","0,47","0,181","7,70%","0,181","0,46","2,19","2,48",...,"3,77","8,27%","6,19%","2,38","10,88%","15,75%","5.417.300,00","1.651.110.000,00","0,72","19,36%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,LWSA3,"3,12","128,42","0,63","1,299","2,44%","0,440","6,54","15,35","13,24",...,"5,67","8,46%","1,01%","1,25","3,27%","0,49%","20.480.500,00","2.800.760.000,00","0,03","31,96%"
173,PETZ3,"4,49","385,69","1,22","0,636","6,48%","0,521","3,64","12,49","-2,48",...,"4,03","5,10%","0,17%","1,65","5,31%","0,32%","38.629.500,00","1.698.680.000,00","0,26","18,82%"
174,PRNR3,"16,20","414,28","2,29","0,786","0,45%","0,634","3,41","12,12","-3,33",...,"9,33","6,48%","0,31%","1,56","6,64%","0,55%","5.389.480,00","330.381.000,00","1,37","43,00%"
175,CRFB3,"6,21","523,89","0,66","0,115","1,53%","0,148","-7,45","3,09","-0,49",...,"5,16","3,72%","0,21%","0,96","6,04%","0,13%","88.418.800,00","19.842.000.000,00","1,12","15,16%"


### 3.5 Selecionando Variáveis para Análise

In [38]:
acoes_df =papel_df[["Papel", "Cotação","P/L", "P/VP", "Div.Yield", "EV/EBIT", "Mrg Líq.", "ROE"]]
acoes_df

,Papel,Cotação,P/L,P/VP,Div.Yield,EV/EBIT,Mrg Líq.,ROE
0,OIBR3,"1,45","0,04","-0,03","0,00%","-2,62","137,65%","-81,27%"
1,AMER3,"5,97","0,11","0,21","0,00%","-1,23","76,26%","196,01%"
2,SYNE3,"5,07","1,57","0,49","129,21%","1,81","44,92%","31,25%"
3,VBBR3,"16,48","2,01","0,90","9,53%","5,50","5,33%","44,82%"
4,POSI3,"5,43","2,96","0,47","7,70%","4,38","6,19%","15,75%"
...,...,...,...,...,...,...,...,...
172,LWSA3,"3,12","128,42","0,63","2,44%","11,94","1,01%","0,49%"
173,PETZ3,"4,49","385,69","1,22","6,48%","12,26","0,17%","0,32%"
174,PRNR3,"16,20","414,28","2,29","0,45%","16,30","0,31%","0,55%"
175,CRFB3,"6,21","523,89","0,66","1,53%","7,42","0,21%","0,13%"


### Metadados:

>  - Papel: ticker do ativo
>  - Cotação: valor atual do ativo em reais
>  - P/L: preço sobre o lucro. Indica quanto o mercado está disposto a pagar pelos lucros gerados pela empresa
>  - P/VP: Preço sobre Valor Patrimonial, Um valor abaixo de 1 pode sugerir que a empresa está sendo negociada por menos do que seu valor patrimonial, indicando uma possível subvalorização.
>  - Div.Yield: Dividend Yield, é o indicador que avalia o desempenho da empresa com base nos proventos pagos aos acionistas nos últimos 12 meses.
>  - EV/EBIT: valor da empresa (Enterprise Value) com o seu lucro antes de juros e impostos (EBIT) 
>  - Mrg Líq.: Margem Líquida, indica quanto da receita é  convertida em lucro
>  - ROE: Retorno sobre o Patrimônio Líquido, é um indicador de rentabilidade que avalia a eficiência de uma empresa em gerar lucro a partir dos seus recursos próprios

In [39]:
# Obter uma descrição dos dados

acoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Papel      177 non-null    object
 1   Cotação    177 non-null    object
 2   P/L        177 non-null    object
 3   P/VP       177 non-null    object
 4   Div.Yield  177 non-null    object
 5   EV/EBIT    177 non-null    object
 6   Mrg Líq.   177 non-null    object
 7   ROE        177 non-null    object
dtypes: object(8)
memory usage: 11.2+ KB


### 3.6 Convertendo Variáveis de String para Float (Exceto a Variável "Papel")

In [40]:
   
acoes_df1 = acoes_df.copy()

acoes_df1["Div.Yield"]=acoes_df1["Div.Yield"].str.slice(0,4)
acoes_df1 = acoes_df1.rename(columns={'Div.Yield': 'Div.Yield %'})

acoes_df1["Mrg Líq."]=acoes_df1["Mrg Líq."].str.slice(0,4)
acoes_df1 = acoes_df1.rename(columns={'Mrg Líq.': 'Mrg Líq. %'})

acoes_df1["ROE"]=acoes_df1["ROE"].str.slice(0,4)
acoes_df1 = acoes_df1.rename(columns={'ROE': 'ROE %'})

acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
0,OIBR3,"1,45","0,04","-0,03","0,00","-2,62","137,","-81,"
1,AMER3,"5,97","0,11","0,21","0,00","-1,23","76,2","196,"
2,SYNE3,"5,07","1,57","0,49","129,","1,81","44,9","31,2"
3,VBBR3,"16,48","2,01","0,90","9,53","5,50","5,33","44,8"
4,POSI3,"5,43","2,96","0,47","7,70","4,38","6,19","15,7"
...,...,...,...,...,...,...,...,...
172,LWSA3,"3,12","128,42","0,63","2,44","11,94","1,01","0,49"
173,PETZ3,"4,49","385,69","1,22","6,48","12,26","0,17","0,32"
174,PRNR3,"16,20","414,28","2,29","0,45","16,30","0,31","0,55"
175,CRFB3,"6,21","523,89","0,66","1,53","7,42","0,21","0,13"


In [41]:
def aux(x):
    return float(x.replace(".","").replace(",", "."))

acoes_df1['Cotação'] = acoes_df1['Cotação'].apply(aux)
acoes_df1['P/L'] = acoes_df1['P/L'].apply(aux)
acoes_df1['P/VP'] = acoes_df1['P/VP'].apply(aux)
acoes_df1['Div.Yield %'] = acoes_df1['Div.Yield %'].apply(aux)
acoes_df1['EV/EBIT'] = acoes_df1['EV/EBIT'].apply(aux)

acoes_df1['Mrg Líq. %'] = acoes_df1['Mrg Líq. %'].apply(aux)
acoes_df1['ROE %'] = acoes_df1['ROE %'].apply(aux)

acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
0,OIBR3,1.45,0.04,-0.03,0.00,-2.62,137.00,-81.00
1,AMER3,5.97,0.11,0.21,0.00,-1.23,76.20,196.00
2,SYNE3,5.07,1.57,0.49,129.00,1.81,44.90,31.20
3,VBBR3,16.48,2.01,0.90,9.53,5.50,5.33,44.80
4,POSI3,5.43,2.96,0.47,7.70,4.38,6.19,15.70
...,...,...,...,...,...,...,...,...
172,LWSA3,3.12,128.42,0.63,2.44,11.94,1.01,0.49
173,PETZ3,4.49,385.69,1.22,6.48,12.26,0.17,0.32
174,PRNR3,16.20,414.28,2.29,0.45,16.30,0.31,0.55
175,CRFB3,6.21,523.89,0.66,1.53,7.42,0.21,0.13


In [42]:
acoes_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Papel        177 non-null    object 
 1   Cotação      177 non-null    float64
 2   P/L          177 non-null    float64
 3   P/VP         177 non-null    float64
 4   Div.Yield %  177 non-null    float64
 5   EV/EBIT      177 non-null    float64
 6   Mrg Líq. %   177 non-null    float64
 7   ROE %        177 non-null    float64
dtypes: float64(7), object(1)
memory usage: 11.2+ KB


### 3.7 Selecionando um Ticker para Cada Empresa Negociada na B3 (menor EV/EBIT por exemplo)


In [43]:
ticker = []

for indice, linha in acoes_df1.iterrows():
    ticker.append(linha["Papel"][0:4])
    
acoes_df1["Empresa"] = ticker

acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,Empresa
0,OIBR3,1.45,0.04,-0.03,0.00,-2.62,137.00,-81.00,OIBR
1,AMER3,5.97,0.11,0.21,0.00,-1.23,76.20,196.00,AMER
2,SYNE3,5.07,1.57,0.49,129.00,1.81,44.90,31.20,SYNE
3,VBBR3,16.48,2.01,0.90,9.53,5.50,5.33,44.80,VBBR
4,POSI3,5.43,2.96,0.47,7.70,4.38,6.19,15.70,POSI
...,...,...,...,...,...,...,...,...,...
172,LWSA3,3.12,128.42,0.63,2.44,11.94,1.01,0.49,LWSA
173,PETZ3,4.49,385.69,1.22,6.48,12.26,0.17,0.32,PETZ
174,PRNR3,16.20,414.28,2.29,0.45,16.30,0.31,0.55,PRNR
175,CRFB3,6.21,523.89,0.66,1.53,7.42,0.21,0.13,CRFB


In [44]:
acoes_df1 = acoes_df1.sort_values(by="EV/EBIT")
acoes_df1 = acoes_df1.drop_duplicates("Empresa")
acoes_df1 = acoes_df1.drop('Empresa', axis=1)
acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
126,CXSE3,14.35,12.22,3.29,9.70,-346.35,0.00,26.90
7,BRAP4,16.75,3.65,0.75,13.60,-263.50,0.00,20.40
0,OIBR3,1.45,0.04,-0.03,0.00,-2.62,137.00,-81.00
1,AMER3,5.97,0.11,0.21,0.00,-1.23,76.20,196.00
56,SANB3,12.51,7.20,1.04,6.13,0.00,0.00,14.40
...,...,...,...,...,...,...,...,...
162,ORVR3,42.65,22.12,4.51,0.00,21.28,20.00,20.40
18,LOGG3,19.00,4.86,0.47,13.30,22.20,156.00,9.58
170,WEGE3,53.94,37.16,11.48,1.40,27.49,17.70,30.80
65,USIM3,5.50,7.56,0.29,4.68,46.93,4.18,3.82


## 4. Fórmula de Benjamin Graham

Benjamin Graham, estabeleceu dois critérios fundamentais para avaliar o preço justo de uma ação:

1. Relação Preço/Lucro (P/L): O preço de uma ação não deve exceder 15 vezes o lucro médio gerado nos últimos três anos.

2. Relação Preço/Valor Patrimonial (P/VP):O preço por ação, dividido pelo valor patrimonial por ação, não deve ser maior que 1.5.

Graham também propôs que o produto dessas duas relações (P/L e P/VP) não deve exceder 22.5.

A fórmula de Graham, leva em consideração métricas como Lucro por Ação (LPA) e Valor Patrimonial por Ação (VPA).

$$preço\ justo = \sqrt{22.5*LPA*VPA}$$

- Por levar em conta o valor patrimonial em seu cálculo, a fórmula de Benjamin Graham apresenta limitações quando aplicada a empresas de tecnologia.

- Para bancos e seguradoras, pode-se utilizar a fórmula adaptada de Joel Greenblatt.








### 4.1 Aplicando os Critérios de Benjamin Graham ao DataFrame

- P/L <= 15

- P/PV > 0 e P/PV <=1.5

- EV/EBIT <= 10


In [45]:
acoes_df2 = acoes_df1[(acoes_df1['P/L'] <= 15) & (acoes_df1['P/VP'] > 0) & (acoes_df1['P/VP'] <= 1.5) & (acoes_df1['EV/EBIT'] > 0) &  (acoes_df1['EV/EBIT'] <= 10)]
acoes_df2

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
2,SYNE3,5.07,1.57,0.49,129.00,1.81,44.90,31.20
32,GOAU4,9.35,5.68,0.49,4.97,1.93,7.53,8.70
29,PETR4,36.58,5.57,1.20,21.70,3.18,16.90,21.50
136,ALOS3,19.10,13.55,0.74,8.79,3.60,32.00,5.49
125,VTRU3,6.20,12.01,0.35,0.00,3.61,3.27,2.92
...,...,...,...,...,...,...,...,...
8,HBOR3,1.47,3.69,0.14,6.19,9.28,12.70,3.73
83,AGRO3,22.01,8.06,1.03,7.07,9.38,22.20,12.70
12,JHSF3,3.87,4.27,0.50,9.56,9.49,43.20,11.70
142,YDUQ3,10.50,14.75,0.94,2.79,9.50,3.90,6.40


### 4.2 Extraindo LPA e VPA de Cada Ativo do Site "Fundamentus"

In [46]:
lista_ativos = list(acoes_df2["Papel"])
lista_ativos

['SYNE3',
 'GOAU4',
 'PETR4',
 'ALOS3',
 'VTRU3',
 'EVEN3',
 'CSED3',
 'HBRE3',
 'VALE3',
 'CEAB3',
 'MDNE3',
 'LREN3',
 'POSI3',
 'MGLU3',
 'VLID3',
 'LJQQ3',
 'CMIG4',
 'SCAR3',
 'ECOR3',
 'RAPT4',
 'FIQE3',
 'SAPR4',
 'VBBR3',
 'ISAE4',
 'BMOB3',
 'GUAR3',
 'RECV3',
 'DESK3',
 'GGBR4',
 'UGPA3',
 'TUPY3',
 'PGMN3',
 'ARML3',
 'VAMO3',
 'MILS3',
 'MYPK3',
 'CSMG3',
 'ENGI11',
 'NEOE3',
 'SOJA3',
 'LAVV3',
 'MELK3',
 'ALUP11',
 'MDIA3',
 'JSLG3',
 'PFRM3',
 'SBFG3',
 'BLAU3',
 'FLRY3',
 'CSAN3',
 'OPCT3',
 'CYRE3',
 'RANI3',
 'CPLE3',
 'HYPE3',
 'CAML3',
 'GRND3',
 'HBOR3',
 'AGRO3',
 'JHSF3',
 'YDUQ3',
 'FESA4']

In [47]:
VPA_list = []
LPA_list = []

for papel in lista_ativos:
    navegador.find_element(By.XPATH, '//*[@id="completar"]').send_keys(papel)    
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[1]/form/fieldset/input[2]').click()
    
    VPA = navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[6]/span').text
    VPA = float(VPA.replace(",","."))
    VPA_list.append(VPA)
    
    LPA = navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[2]/td[6]/span').text
    LPA = float(LPA.replace(",","."))         
    LPA_list.append(LPA)

### 4.3 Adicionando as Colunas LPA e VPA ao DataFrame

In [48]:
acoes_df3 = acoes_df2.copy()
acoes_df3["VPA"]= VPA_list
acoes_df3["LPA"]= LPA_list

acoes_df3

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA
2,SYNE3,5.07,1.57,0.49,129.00,1.81,44.90,31.20,10.31,3.22
32,GOAU4,9.35,5.68,0.49,4.97,1.93,7.53,8.70,18.91,1.65
29,PETR4,36.58,5.57,1.20,21.70,3.18,16.90,21.50,30.54,6.57
136,ALOS3,19.10,13.55,0.74,8.79,3.60,32.00,5.49,25.68,1.41
125,VTRU3,6.20,12.01,0.35,0.00,3.61,3.27,2.92,17.66,0.52
...,...,...,...,...,...,...,...,...,...,...
8,HBOR3,1.47,3.69,0.14,6.19,9.28,12.70,3.73,10.68,0.40
83,AGRO3,22.01,8.06,1.03,7.07,9.38,22.20,12.70,21.45,2.73
12,JHSF3,3.87,4.27,0.50,9.56,9.49,43.20,11.70,7.73,0.91
142,YDUQ3,10.50,14.75,0.94,2.79,9.50,3.90,6.40,11.13,0.71


### 4.4Cálculo do Valor Justo

In [49]:
acoes_df3['Preco_Justo'] = round((22.5*acoes_df3['VPA']*acoes_df3['LPA'])**0.5, 2)

acoes_df3

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo
2,SYNE3,5.07,1.57,0.49,129.00,1.81,44.90,31.20,10.31,3.22,27.33
32,GOAU4,9.35,5.68,0.49,4.97,1.93,7.53,8.70,18.91,1.65,26.50
29,PETR4,36.58,5.57,1.20,21.70,3.18,16.90,21.50,30.54,6.57,67.19
136,ALOS3,19.10,13.55,0.74,8.79,3.60,32.00,5.49,25.68,1.41,28.54
125,VTRU3,6.20,12.01,0.35,0.00,3.61,3.27,2.92,17.66,0.52,14.37
...,...,...,...,...,...,...,...,...,...,...,...
8,HBOR3,1.47,3.69,0.14,6.19,9.28,12.70,3.73,10.68,0.40,9.80
83,AGRO3,22.01,8.06,1.03,7.07,9.38,22.20,12.70,21.45,2.73,36.30
12,JHSF3,3.87,4.27,0.50,9.56,9.49,43.20,11.70,7.73,0.91,12.58
142,YDUQ3,10.50,14.75,0.94,2.79,9.50,3.90,6.40,11.13,0.71,13.33


### 4.5 Margens de Segurança

In [50]:
acoes_df3['Potencial rentabilidad %'] = round((acoes_df3['Preco_Justo']/acoes_df3['Cotação']-1)*100, 2)
acoes_df3['Margem seguranca 60%'] = round(acoes_df3['Preco_Justo']*0.4, 2)
acoes_df3['Margem seguranca 50%'] = round(acoes_df3['Preco_Justo']*0.5, 2)
acoes_df3['Margem seguranca 40%'] = round(acoes_df3['Preco_Justo']*0.6, 2)

acoes_df3

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
2,SYNE3,5.07,1.57,0.49,129.00,1.81,44.90,31.20,10.31,3.22,27.33,439.05,10.93,13.66,16.40
32,GOAU4,9.35,5.68,0.49,4.97,1.93,7.53,8.70,18.91,1.65,26.50,183.42,10.60,13.25,15.90
29,PETR4,36.58,5.57,1.20,21.70,3.18,16.90,21.50,30.54,6.57,67.19,83.68,26.88,33.60,40.31
136,ALOS3,19.10,13.55,0.74,8.79,3.60,32.00,5.49,25.68,1.41,28.54,49.42,11.42,14.27,17.12
125,VTRU3,6.20,12.01,0.35,0.00,3.61,3.27,2.92,17.66,0.52,14.37,131.77,5.75,7.18,8.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,HBOR3,1.47,3.69,0.14,6.19,9.28,12.70,3.73,10.68,0.40,9.80,566.67,3.92,4.90,5.88
83,AGRO3,22.01,8.06,1.03,7.07,9.38,22.20,12.70,21.45,2.73,36.30,64.93,14.52,18.15,21.78
12,JHSF3,3.87,4.27,0.50,9.56,9.49,43.20,11.70,7.73,0.91,12.58,225.06,5.03,6.29,7.55
142,YDUQ3,10.50,14.75,0.94,2.79,9.50,3.90,6.40,11.13,0.71,13.33,26.95,5.33,6.66,8.00


#### 4.5.1 Ativos com Margem de Segurança de Pelo Menos 60%

In [51]:
acoes60 = acoes_df3[acoes_df3['Cotação'] < acoes_df3['Margem seguranca 60%']]
acoes60

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
2,SYNE3,5.07,1.57,0.49,129.00,1.81,44.90,31.20,10.31,3.22,27.33,439.05,10.93,13.66,16.40
32,GOAU4,9.35,5.68,0.49,4.97,1.93,7.53,8.70,18.91,1.65,26.50,183.42,10.60,13.25,15.90
5,HBRE3,2.96,3.10,0.15,0.00,3.76,146.00,4.94,19.31,0.95,20.32,586.49,8.13,10.16,12.19
13,MDNE3,12.11,4.28,0.66,5.37,3.97,16.10,15.40,18.30,2.83,34.14,181.92,13.66,17.07,20.48
4,POSI3,5.43,2.96,0.47,7.70,4.38,6.19,15.70,11.64,1.83,21.89,303.13,8.76,10.94,13.13
11,SCAR3,17.77,4.11,0.61,19.60,5.26,22.00,14.90,28.93,4.32,53.03,198.42,21.21,26.52,31.82
3,VBBR3,16.48,2.01,0.90,9.53,5.50,5.33,44.80,18.25,8.18,57.96,251.70,23.18,28.98,34.78
14,CAML3,4.39,4.41,0.43,6.68,8.69,2.92,9.80,10.15,0.99,15.04,242.60,6.02,7.52,9.02
8,HBOR3,1.47,3.69,0.14,6.19,9.28,12.70,3.73,10.68,0.40,9.80,566.67,3.92,4.90,5.88
12,JHSF3,3.87,4.27,0.50,9.56,9.49,43.20,11.70,7.73,0.91,12.58,225.06,5.03,6.29,7.55


#### 4.5.2 Ativos com Margem de Segurança de Pelo Menos 50%

In [52]:
acoes50 = acoes_df3[(acoes_df3['Cotação'] <= acoes_df3['Margem seguranca 50%']) & (acoes_df3['Cotação'] > acoes_df3['Margem seguranca 60%'])]
acoes50

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
125,VTRU3,6.20,12.01,0.35,0.00,3.61,3.27,2.92,17.66,0.52,14.37,131.77,5.75,7.18,8.62
10,CMIG4,10.97,3.92,1.12,9.21,5.25,20.70,28.50,9.81,2.80,24.86,126.62,9.94,12.43,14.92
9,ECOR3,4.83,3.82,0.96,4.03,5.28,9.29,25.10,5.03,1.26,11.94,147.20,4.78,5.97,7.16
25,SAPR4,5.38,5.43,0.77,5.87,5.49,22.10,14.10,6.99,0.99,12.48,131.97,4.99,6.24,7.49
17,ISAE4,23.90,4.76,0.80,3.29,5.58,47.10,16.70,29.94,5.02,58.15,143.31,23.26,29.08,34.89
52,GGBR4,16.70,7.10,0.62,4.82,5.70,7.49,8.73,26.94,2.35,37.74,125.99,15.10,18.87,22.64
42,VAMO3,4.06,6.32,0.84,6.75,5.99,9.87,13.30,4.81,0.64,8.32,104.93,3.33,4.16,4.99
111,MYPK3,13.24,10.78,0.49,4.57,6.16,2.01,4.56,26.94,1.23,27.31,106.27,10.92,13.66,16.39
33,NEOE3,18.20,5.88,0.68,3.70,6.43,8.12,11.50,26.80,3.09,43.17,137.20,17.27,21.58,25.90
40,JSLG3,5.56,6.18,0.86,7.56,7.15,2.95,13.80,6.49,0.90,11.46,106.12,4.58,5.73,6.88


#### 4.5.3 Ativos com Margem de Segurança de Pelo Menos 40%

In [53]:
acoes40 = acoes_df3[(acoes_df3['Cotação'] <= acoes_df3['Margem seguranca 40%']) & (acoes_df3['Cotação'] > acoes_df3['Margem seguranca 50%'])]
acoes40

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
29,PETR4,36.58,5.57,1.20,21.70,3.18,16.90,21.50,30.54,6.57,67.19,83.68,26.88,33.60,40.31
107,EVEN3,5.94,10.49,0.64,15.20,3.65,10.10,6.09,9.30,0.57,10.92,83.84,4.37,5.46,6.55
74,CSED3,3.48,7.88,0.86,4.73,3.69,6.43,10.80,4.06,0.44,6.34,82.18,2.54,3.17,3.80
21,VALE3,54.83,5.16,1.19,9.76,3.78,22.80,23.10,45.95,10.63,104.83,91.19,41.93,52.42,62.90
84,CEAB3,9.65,8.12,0.94,3.57,3.91,4.96,11.50,10.32,1.19,16.62,72.23,6.65,8.31,9.97
138,MGLU3,7.07,14.27,0.47,0.00,4.64,0.97,3.31,14.97,0.50,12.98,83.59,5.19,6.49,7.79
28,VLID3,23.90,5.53,1.16,7.44,4.72,15.90,20.90,20.64,4.32,44.79,87.41,17.92,22.40,26.87
82,LJQQ3,2.23,8.05,0.77,5.47,5.09,2.08,9.58,2.89,0.28,4.27,91.48,1.71,2.13,2.56
86,RAPT4,8.58,8.16,0.92,4.24,5.28,5.49,11.20,9.35,1.05,14.86,73.19,5.94,7.43,8.92
49,FIQE3,3.37,7.01,1.04,7.56,5.31,17.50,14.90,3.23,0.48,5.91,75.37,2.36,2.96,3.55


### 5. Ranking dos Ativos

- Para realizar o ranking, os indicadores a serem considerados são: Potencial de Rentabilidade, Dividend Yield, Margem Líquida e ROE. Todos esses indicadores têm melhor desempenho quanto maiores forem os seus valores.

In [54]:
Rank_df = acoes_df3[acoes_df3['Cotação'] <= acoes_df3['Margem seguranca 40%']]

Ranking1 =Rank_df[["Papel", "Cotação","Potencial rentabilidad %","EV/EBIT", "Div.Yield %", "Mrg Líq. %", "ROE %"]]

Ranking = Ranking1.copy()

Ranking['PR_rank'] = Ranking['Potencial rentabilidad %'].rank(ascending=False)

Ranking['EV/EBIT_rank'] = Ranking['EV/EBIT'].rank()

Ranking['DY_rank'] = Ranking['Div.Yield %'].rank(ascending=False)

Ranking['ML_rank'] = Ranking['Mrg Líq. %'].rank(ascending=False)

Ranking['ROE_rank'] = Ranking['ROE %'].rank(ascending=False)

Ranking['suma_rank'] = Ranking['PR_rank'] + Ranking['DY_rank'] + Ranking['ML_rank'] + Ranking['ROE_rank']
Ranking = Ranking.sort_values(by="suma_rank")

Ranking



,Papel,Cotação,Potencial rentabilidad %,EV/EBIT,Div.Yield %,Mrg Líq. %,ROE %,PR_rank,EV/EBIT_rank,DY_rank,ML_rank,ROE_rank,suma_rank
2,SYNE3,5.07,439.05,1.81,129.00,44.90,31.20,3.0,1.0,1.0,3.0,2.0,9.0
11,SCAR3,17.77,198.42,5.26,19.60,22.00,14.90,8.0,16.0,3.0,8.0,17.5,36.5
10,CMIG4,10.97,126.62,5.25,9.21,20.70,28.50,19.0,15.0,12.0,10.0,3.0,44.0
12,JHSF3,3.87,225.06,9.49,9.56,43.20,11.70,7.0,43.0,9.0,4.0,24.0,44.0
21,VALE3,54.83,91.19,3.78,9.76,22.80,23.10,26.0,8.0,8.0,6.0,5.0,45.0
3,VBBR3,16.48,251.70,5.50,9.53,5.33,44.80,5.0,21.0,10.0,33.0,1.0,49.0
29,PETR4,36.58,83.68,3.18,21.70,16.90,21.50,31.0,3.0,2.0,16.0,6.5,55.5
31,LAVV3,8.96,81.36,6.52,7.80,26.50,21.50,34.0,33.0,14.0,5.0,6.5,59.5
41,CSMG3,23.26,77.94,6.21,11.90,17.80,17.80,35.0,30.0,6.0,13.0,9.0,63.0
17,ISAE4,23.90,143.31,5.58,3.29,47.10,16.70,12.0,22.0,37.0,2.0,12.0,63.0


## 5. Conclusão

- Antes de montar uma carteira de ativos, é fundamental analisar cada empresa por meio de seus relatórios financeiros e de desempenho. Isso permite construir uma carteira diversificada, selecionando os ativos com base no princípio da margem de segurança ou utilizando um ranking que combina diferentes indicadores financeiros.